# 0.0. Planejamento da Solução ( IOT )

## Input - Entrada

### O problema de negócio

1. Ajudar a desenvolver o mercado artesanal e independnete de bolos e doces.
2. Quais cidades brasileiras a Moça deveria iniciar um programa de incentivo ao empreendedorismo gastronômico artesenal.
3. A proposta deverá considerar a sustentabilidade e escabilidade do negócio.

## Output - Saída

1. Apresentar um parâmetro do que seria o mercado artesanal e independente de bolos e doces.
2. Apresentar indicadores que deveríamos considerar para que uma cidade seja potencial escolhida como um local para incentivo ao empreendedorismo gastronômico.
3. Indicar em termos de receita qual seria o potencial do local escolhido e porquê ele é importante.

## Tasks - Processo

1. **Apresentar um parâmetro do que seria o mercado artesanal e independente de bolos e doces.**
    * Qual é o contexto do mercado artesanal de bolos e doces no Brasil ?
            - Como é composto este mercado (sexo, idade, região)
            - Quais os principais mercados
            - Qual é o potencial em cada região
            - Há um grande volume de lojas pequenas de rua com "bolos caseiros". São potenciais fornecedores para a plataforma Vem de Bolo?
            - Pessoas que vendem pela internet (facebook, whatsapp, instagram). Qual o potencial deste mercado?
            - Quais os principais empecilhos para este tipo de mercado? Uma plataforma on-line (Vem de bolo) poderia dar uma escabilidade aos negócios destas pessoas?
            - Qual é a melhor maneira de atingir o público em termos de comunicação?
            
            
2. **Apresentar indicadores que deveríamos considerar para que uma cidade seja potencial escolhida como um local para incentivo ao empreendedorismo gastronômico.**
    * Quais são os fatores relevantes que a Vem de Bolo pode nos ensinar?
         - **Aprendizado via fornecedor:**
            - Qual é o faturamento médio por fonecedor?
            - Como estes fornecedores estão distribuídos? (cidade/localização)
            - O que caracteriza um fornecedor portencial fornecedor para a plataforma Vem de Bolo? Quais são os atributos desejáveis?
            - Quais as principais formas de entrega por fornecedor?
         - **Aprendizado via clientes:**
            - Qual é o perfil de compra das pessoas que utilizam a plataforma Vem de Bolo?
            - Qual é o valor de desembolso médio das pessoas?
            - Qual é o tipo de produto que as pessoas mais compram pela plataforma? Isto está relacionado a escolha ou ao viés da oferta?
            - O que determina a escolha do cliente para dado fornecedor? O rating é algo importante / fator de escolha?
    
    
3. **3. Indicar em termos de receita qual seria o potencial do local escolhido e porquê ele é importante.**
    * Dentre os indicadores socio-econômicos, o que faz um local ser viável ao negócio Vem de Bolo?
            - Qual seria o potencial de ganho na cidade escolhida?
            - Há escabilidade onde poderíamos aumentar a margem sem incorrer de grandes custos (adotar uma cidade que poderia ser um "satélite" de uma região
            - Quais seriam os custos que a Moça deveria investir para viabilizar o negócio da plataforma (marketing, negócio, TI)?

# 0.0. Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.core.display import HTML

## 0.1. Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

def describe_num(num_attributes):
    a = num_attributes.describe().T.reset_index()
    
    d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attributes.apply( lambda x: x.skew())).T      
    d6 = pd.DataFrame(num_attributes.apply( lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(num_attributes.apply( lambda x: x.unique().shape[0])).T
    
    n = pd.concat([d4, d5, d6, d7], axis=0).T.reset_index()
    n.columns = ['attibutes', 'range', 'skew', 'kurtosis', 'unique']
    b = a.merge(n, right_on='attibutes', left_on='index')
    b.drop('index', axis=1, inplace=True)
    columns = ['attibutes', 'count', 'mean', 'std', '25%', '50%', '75%', 'min', 'max', 'range',
           'skew', 'kurtosis', 'unique']
    return b[columns]

Populating the interactive namespace from numpy and matplotlib


## 0.2. Load dataset

In [3]:
df_addresses = pd.read_csv('../data/raw/tb_addresses.csv', delimiter=';')
df_banners = pd.read_csv('../data/raw/tb_banners.csv', delimiter=';')
df_cities = pd.read_csv('../data/raw/tb_cities.csv', delimiter=';')
df_coupons = pd.read_csv('../data/raw/tb_coupons.csv', delimiter=';')
df_customer_payments = pd.read_csv('../data/raw/tb_customer_payments.csv', delimiter=';')
df_ordered_products = pd.read_csv('../data/raw/tb_ordered_products.csv', delimiter=';')
df_product_tags = pd.read_csv('../data/raw/tb_product_tags.csv', delimiter=';')
df_products = pd.read_csv('../data/raw/tb_products.csv', delimiter=';')
df_ratings = pd.read_csv('../data/raw/tb_ratings.csv', delimiter=';')
df_solds_cognatis = pd.read_csv('../data/raw/tb_solds_cognatis.csv', delimiter=';')
df_users = pd.read_csv('../data/raw/tb_users.csv', delimiter=';')
df_vendor_payments = pd.read_csv('../data/raw/tb_vendor_payments.csv', delimiter=';')
df_vendors = pd.read_csv('../data/raw/tb_vendors.csv', delimiter=';')
df_vendors_delivery_fees = pd.read_csv('../data/raw/tb_vendors_delivery_fees.csv', delimiter=';')

### DF VENDORS T = DF VENDORS + DF VENDORS DELIVERY FEES + DF ADRESSES

#### AJUSTANDO - DF VENDORS DELIVERY FEES

In [68]:
# TIPO TAXA FIXED
# Ajustando o df_vendors_delivery_fees pelo fato de possuir dados repetidos na chave principal dos fornecedores de maneira a criar uma única tabela.
fixed = df_vendors_delivery_fees[df_vendors_delivery_fees['tipo_taxa'] == 'FIXED'].reset_index().drop('index', axis=1)
fixed = fixed.rename(columns={'valor_fixo': 'fixed_valor_fixo', 'distancia_maxima': 'fixed_distancia_maxima'})
fixed = fixed.drop(['valor_minimo', 'valor_por_km', 'valor_cobertura_minima', 'tipo_taxa'], axis=1)

# Agrupando o dado dos fornecedores num único registro.
id_fornecedor = fixed['id_fornecedor'].unique().tolist()
fixed_ndp = fixed.drop_duplicates(subset='id_fornecedor').copy().reset_index().drop('index', axis=1)
fixed_ndp['fixed_valor_fixo'] = ""
fixed_ndp['fixed_distancia_maxima'] = ""
fixed['fixed_valor_fixo'] = fixed['fixed_valor_fixo'].astype('int64')
fixed['fixed_valor_fixo'] = fixed['fixed_valor_fixo'].astype('str')
fixed['fixed_distancia_maxima'] = fixed['fixed_distancia_maxima'].astype('str')
for i in range(len(fixed_ndp)):
    for j in range(len(fixed)):
        if fixed_ndp['id_fornecedor'][i] == fixed['id_fornecedor'][j]:
            fixed_ndp['fixed_valor_fixo'][i] = fixed_ndp['fixed_valor_fixo'][i] + fixed['fixed_valor_fixo'][j] + ' / '
for i in range(len(fixed_ndp)):
    for j in range(len(fixed)):
        if fixed_ndp['id_fornecedor'][i] == fixed['id_fornecedor'][j]:
            fixed_ndp['fixed_distancia_maxima'][i] = fixed_ndp['fixed_distancia_maxima'][i] + fixed['fixed_distancia_maxima'][j] + ' / '
fixed = fixed_ndp.copy()

fixed.shape

(107, 3)

In [69]:
# TIPO TAXA PER_KM
# Ajustando o df_vendors_delivery_fees pelo fato de possuir dados repetidos na chave principal dos fornecedores de maneira a criar uma única tabela.
per_km = df_vendors_delivery_fees[df_vendors_delivery_fees['tipo_taxa'] == 'PER_KM'].reset_index().drop('index', axis=1)
per_km = per_km.rename(columns={'valor_por_km': 'per_km_valor', 'distancia_maxima': 'per_km_distancia_maxima'})
per_km = per_km.drop(['valor_fixo', 'valor_minimo', 'valor_cobertura_minima', 'tipo_taxa'], axis=1)

# Agrupando o dado dos fornecedores num único registro.
id_fornecedor = per_km['id_fornecedor'].unique().tolist()
per_km_ndp = per_km.drop_duplicates(subset='id_fornecedor').copy().reset_index().drop('index', axis=1)
per_km_ndp['per_km_valor'] = ""
per_km_ndp['per_km_distancia_maxima'] = ""
per_km['per_km_valor'] = per_km['per_km_valor'].astype('str')
per_km['per_km_distancia_maxima'] = per_km['per_km_distancia_maxima'].astype('str')
for i in range(len(per_km_ndp)):
    for j in range(len(per_km)):
        if per_km_ndp['id_fornecedor'][i] == per_km['id_fornecedor'][j]:
            per_km_ndp['per_km_valor'][i] = per_km_ndp['per_km_valor'][i] + per_km['per_km_valor'][j] + ' / '
for i in range(len(per_km_ndp)):
    for j in range(len(per_km)):
        if per_km_ndp['id_fornecedor'][i] == per_km['id_fornecedor'][j]:
            per_km_ndp['per_km_distancia_maxima'][i] = per_km_ndp['per_km_distancia_maxima'][i] + per_km['per_km_distancia_maxima'][j] + ' / '
per_km = per_km_ndp.copy()

# MERGING - FIXED + PER_KM
n_df_vendors_delivery = fixed.merge(per_km, on='id_fornecedor', how='left')
n_df_vendors_delivery.isna().sum()

id_fornecedor               0
fixed_valor_fixo            0
fixed_distancia_maxima      0
per_km_valor               88
per_km_distancia_maxima    88
dtype: int64

In [70]:
# TIPO TAXA MIN PLUS PER KM
# Ajustando o df_vendors_delivery_fees pelo fato de possuir dados repetidos na chave principal dos fornecedores de maneira a criar uma única tabela.
min_plus_per_km = df_vendors_delivery_fees[df_vendors_delivery_fees['tipo_taxa'] == 'MIN_PLUS_PER_KM'].reset_index().drop('index', axis=1)
min_plus_per_km = min_plus_per_km.rename(columns={'valor_minimo': 'min_plus_per_km_valor_minimo',
                                                  'valor_por_km': 'min_plus_per_km_valor_por_km',
                                                  'valor_cobertura_minima': 'min_plus_per_km_valor_cobertura_minima',
                                                  'distancia_maxima': 'min_plus_per_km_distancia_maxima',
                                                 })
min_plus_per_km = min_plus_per_km.drop(['valor_fixo', 'tipo_taxa'], axis=1)

# Agrupando o dado dos fornecedores num único registro.
id_fornecedor = min_plus_per_km['id_fornecedor'].unique().tolist()
min_plus_per_km_ndp = min_plus_per_km.drop_duplicates(subset='id_fornecedor').copy().reset_index().drop('index', axis=1)
min_plus_per_km_ndp['min_plus_per_km_valor_minimo'] = ""
min_plus_per_km_ndp['min_plus_per_km_valor_por_km'] = ""
min_plus_per_km_ndp['min_plus_per_km_valor_cobertura_minima'] = ""
min_plus_per_km_ndp['min_plus_per_km_distancia_maxima'] = ""
min_plus_per_km['min_plus_per_km_valor_minimo'] = min_plus_per_km['min_plus_per_km_valor_minimo'].astype('str')
min_plus_per_km['min_plus_per_km_valor_por_km'] = min_plus_per_km['min_plus_per_km_valor_por_km'].astype('str')
min_plus_per_km['min_plus_per_km_valor_cobertura_minima'] = min_plus_per_km['min_plus_per_km_valor_cobertura_minima'].astype('str')
min_plus_per_km['min_plus_per_km_distancia_maxima'] = min_plus_per_km['min_plus_per_km_distancia_maxima'].astype('str')
for i in range(len(min_plus_per_km_ndp)):
    for j in range(len(min_plus_per_km)):
        if min_plus_per_km_ndp['id_fornecedor'][i] == min_plus_per_km['id_fornecedor'][j]:
            min_plus_per_km_ndp['min_plus_per_km_valor_minimo'][i] = min_plus_per_km_ndp['min_plus_per_km_valor_minimo'][i] + min_plus_per_km['min_plus_per_km_valor_minimo'][j] + ' / '
for i in range(len(min_plus_per_km_ndp)):
    for j in range(len(min_plus_per_km)):
        if min_plus_per_km_ndp['id_fornecedor'][i] == min_plus_per_km['id_fornecedor'][j]:
            min_plus_per_km_ndp['min_plus_per_km_valor_por_km'][i] = min_plus_per_km_ndp['min_plus_per_km_valor_por_km'][i] + min_plus_per_km['min_plus_per_km_valor_por_km'][j] + ' / '
for i in range(len(min_plus_per_km_ndp)):
    for j in range(len(min_plus_per_km)):
        if min_plus_per_km_ndp['id_fornecedor'][i] == min_plus_per_km['id_fornecedor'][j]:
            min_plus_per_km_ndp['min_plus_per_km_valor_cobertura_minima'][i] = min_plus_per_km_ndp['min_plus_per_km_valor_cobertura_minima'][i] + min_plus_per_km['min_plus_per_km_valor_cobertura_minima'][j] + ' / '
for i in range(len(min_plus_per_km_ndp)):
    for j in range(len(min_plus_per_km)):
        if min_plus_per_km_ndp['id_fornecedor'][i] == min_plus_per_km['id_fornecedor'][j]:
            min_plus_per_km_ndp['min_plus_per_km_distancia_maxima'][i] = min_plus_per_km_ndp['min_plus_per_km_distancia_maxima'][i] + min_plus_per_km['min_plus_per_km_distancia_maxima'][j] + ' / '
min_plus_per_km = min_plus_per_km_ndp.copy()


# MERGING - N DF VENDORS DELIVERY + MIN PLUS PER_KM
n_df_vendors_delivery = n_df_vendors_delivery.merge(min_plus_per_km, on='id_fornecedor', how='left')
n_df_vendors_delivery.isna().sum()

id_fornecedor                              0
fixed_valor_fixo                           0
fixed_distancia_maxima                     0
per_km_valor                              88
per_km_distancia_maxima                   88
min_plus_per_km_valor_minimo              95
min_plus_per_km_valor_por_km              95
min_plus_per_km_valor_cobertura_minima    95
min_plus_per_km_distancia_maxima          95
dtype: int64

In [71]:
# TIPO TAXA FREE
# Ajustando o df_vendors_delivery_fees pelo fato de possuir dados repetidos na chave principal dos fornecedores de maneira a criar uma única tabela.
free = df_vendors_delivery_fees[df_vendors_delivery_fees['tipo_taxa'] == 'FREE'].reset_index().drop('index', axis=1)
free = free.rename(columns={'distancia_maxima': 'free_distancia_maxima'})
free = free.drop(['valor_fixo', 'valor_minimo','valor_por_km', 'valor_cobertura_minima', 'tipo_taxa'], axis=1)

# Agrupando o dado dos fornecedores num único registro.
id_fornecedor = free['id_fornecedor'].unique().tolist()
free_ndp = free.drop_duplicates(subset='id_fornecedor').copy().reset_index().drop('index', axis=1)
free_ndp['free_distancia_maxima'] = ""
free['free_distancia_maxima'] = free['free_distancia_maxima'].astype('str')
for i in range(len(free_ndp)):
    for j in range(len(free)):
        if free_ndp['id_fornecedor'][i] == free['id_fornecedor'][j]:
            free_ndp['free_distancia_maxima'][i] = free_ndp['free_distancia_maxima'][i] + free['free_distancia_maxima'][j] + ' / '
free = free_ndp.copy()

# MERGING - N DF VENDORS DELIVERY + MIN PLUS PER_KM
n_df_vendors_delivery = n_df_vendors_delivery.merge(free, on='id_fornecedor', how='left')
n_df_vendors_delivery.isna().sum()

id_fornecedor                              0
fixed_valor_fixo                           0
fixed_distancia_maxima                     0
per_km_valor                              88
per_km_distancia_maxima                   88
min_plus_per_km_valor_minimo              95
min_plus_per_km_valor_por_km              95
min_plus_per_km_valor_cobertura_minima    95
min_plus_per_km_distancia_maxima          95
free_distancia_maxima                     86
dtype: int64

In [72]:
# preencher os valores que não são aplicáveis por 0 / criado uma nova coluna para diferenciar quandoe está indicado o valor do delivery para melhorar a filtragem
n_df_vendors_delivery = n_df_vendors_delivery.fillna(0)
n_df_vendors_delivery['preco_delivery'] = 1

In [112]:
df_vendors_delivery_fees.head()

,id_fornecedor,fixed_valor_fixo,fixed_distancia_maxima,per_km_valor,per_km_distancia_maxima,min_plus_per_km_valor_minimo,min_plus_per_km_valor_por_km,min_plus_per_km_valor_cobertura_minima,min_plus_per_km_distancia_maxima,free_distancia_maxima,preco_delivery
0,5f550609cba01d00210ed015,5 /,5 /,0,0,0,0,0,0,0,1
1,5f584189fc58110021464422,9 /,4 /,1.15 /,13 /,0,0,0,0,0,1
2,5f5bc310b8742200213bda1d,5 / 15 / 20 /,5 / 10 / 15 /,0,0,0,0,0,0,0,1
3,5f601c9d70be91002137eb5c,12 /,6 /,0,0,0,0,0,0,0,1
4,5f60ef0070be91002137ebba,8 / 16 / 20 / 23 / 25 /,2 / 4 / 6 / 7 / 8 /,0,0,0,0,0,0,1 /,1


In [73]:
df_vendors_delivery_fees = n_df_vendors_delivery.copy()

#### AJUSTANDO - DF ADRESSES

In [34]:
df_addresses.isna().sum()

id_endereco             0
id_fornecedor       86808
bairro               3762
ponto_referencia    66881
cidade                  0
estado                  0
uf                      0
pais                    0
tipo                    0
cep                  3039
dtype: int64

In [58]:
print('Total de valores únicos', len(df_addresses['id_endereco'].unique()), '/ Total de valores na tabela', df_addresses.shape[0])

Total de valores únicos 87204 / Total de valores na tabela 87217


In [59]:
# Analisando os endereços duplicados, verificou-se que é apenas variação do CEP. Os valores duplicados serão eliminados.
df_addresses = df_addresses.drop_duplicates(subset='id_endereco')
df_addresses.shape

(87204, 10)

#### AJUSTANDO - DF VENDORS

In [61]:
df_vendors.isna().sum()

id_fornecedor         0
nome_fornecedor       0
data_registro         0
qtde_produtos         0
preco_medio           0
avaliacao_media       0
e_aprovado            0
nome_empresa          0
id_endereco           0
delivery_raio_km      0
delivery_sistema    367
hora_abertura         0
hora_fechamento       0
data_aprovacao      209
data_reprovacao     264
dtype: int64

In [62]:
print('Total de valores únicos', len(df_vendors['id_fornecedor'].unique()), '/ Total de valores na tabela', df_vendors.shape[0])

Total de valores únicos 403 / Total de valores na tabela 403


In [64]:
# as colunas abaixo possuem valores 0 em todas as linhas. será retirado do DF.
df_vendors = df_vendors.drop(columns=['qtde_produtos', 'preco_medio', 'avaliacao_media'])

#### JUNTANDO - DF VENDORS + DF VENDORS DELIVERY FEES + DF ADRESSES

In [77]:
# Juntando a tabela DF VENDORS com a VENDORS DELIVERY FEES
df_vendors_t = df_vendors.merge(df_vendors_delivery_fees, on="id_fornecedor", how="left")
df_vendors_t = df_vendors_t.drop(columns='nome_empresa')
# Juntando a tabela DF VENDORS T com a ADRESSES
df_t = df_addresses.drop('id_fornecedor', axis=1) #id_fornecedor está praticamente vazia neste DF.
df_vendors_t = pd.merge(df_vendors_t, df_t, on='id_endereco', how='left')
df_vendors_t = df_vendors_t.drop('id_endereco', axis=1)

In [84]:
dic = { 'data_registro': 'forn_data_registro',
        'qtde_produtos': 'forn_qtde_produtos',
        'preco_medio' : 'forn_preco_medio',
        'avaliacao_media': 'forn_avaliacao_media',
        'e_aprovado': 'forn_e_aprovado',
        'nome_empresa': 'forn_nome_empresa',
        'delivery_raio_km': 'forn_delivery_raio_km',
        'delivery_sistema': 'forn_delivery_sistema',
        'hora_abertura': 'forn_hora_abertura',
        'hora_fechamento': 'forn_hora_fechamento', 
        'data_aprovacao': 'forn_data_aprovacao', 
        'data_reprovacao': 'forn_data_reprovacao', 
        'bairro': 'forn_adress_bairro', 
        'ponto_referencia': 'forn_adress_ponto_referencia', 
        'cidade': 'forn_adress_cidade',
        'estado': 'forn_adress_estado', 
        'uf': 'forn_adress_uf', 
        'pais': 'forn_adress_pais', 
        'tipo': 'forn_adress_tipo_endereco', 
        'cep': 'forn_adress_cep'}
df_vendors_t = df_vendors_t.rename(columns=dic)
df_vendors_t.shape

(403, 28)

In [113]:
df_vendors_t.head()

,id_fornecedor,nome_fornecedor,forc_data_registro,forc_e_aprovado,forc_delivery_raio_km,forc_delivery_sistema,forc_hora_abertura,forc_hora_fechamento,forc_data_aprovacao,forc_data_reprovacao,fixed_valor_fixo,fixed_distancia_maxima,per_km_valor,per_km_distancia_maxima,min_plus_per_km_valor_minimo,min_plus_per_km_valor_por_km,min_plus_per_km_valor_cobertura_minima,min_plus_per_km_distancia_maxima,free_distancia_maxima,preco_delivery,forc_adress_bairro,forc_adress_ponto_referencia,forc_adress_cidade,forc_adress_estado,forc_adress_uf,forc_adress_pais,forc_adress_tipo_endereco,forc_adress_cep
0,5f550609cba01d00210ed015,Meus Bolos,2020-09-06 00:42:34,False,20,THIRD_PARTY,05:30,19:00,2021-06-01 16:22:09,2021-08-10 15:40:47,5 /,5 /,0,0,0,0,0,0,0,1.0,Taquaral,NaN,Campinas,SP,SP,Brasil,HISTORY,13000001.0
1,5f584189fc58110021464422,Moça Doceria,2020-09-09 01:56:25,False,13,NaN,08:00,18:00,2021-02-16 18:25:45,2021-09-09 13:45:56,9 /,4 /,1.15 /,13 /,0,0,0,0,0,1.0,Campo Belo,Ateliê do Doce,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
2,5f5a8be1ccc6c40021a8185f,Cami's Candy,2020-09-10 00:49:18,True,15,NaN,08:00,19:00,2021-08-16 21:06:05,2021-06-25 17:22:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklin,NaN,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
3,5f5bc310b8742200213bda1d,Re doces,2020-09-11 01:07:10,True,15,NaN,10:00,17:00,2020-09-30 17:13:38,NaN,5 / 15 / 20 /,5 / 10 / 15 /,0,0,0,0,0,0,0,1.0,Vila Congonhas,NaN,São Paulo,São Paulo,SP,Brasil,OFFICE,1000001.0
4,5f5bd2fcb8742200213bda24,Doceria Luz,2020-09-11 01:07:10,False,13,THIRD_PARTY,11:00,17:00,2021-04-26 20:55:50,2021-04-26 20:55:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Várzea de Baixo,Edifício 17007 - Open Mall,São Paulo,São Paulo,SP,Brasil,OFFICE,NaN


### DF PRODUCTS T = DF PRODUCTS + DF PRODUCTS TAG + DF VENDORS T

#### AJUSTANDO - DF PRODUCTS TAG

In [86]:
df_product_tags.isna().sum()

tag_produtos_id    0
destacado          0
texto_display      0
tipo               0
dtype: int64

In [87]:
print('Total de valores únicos', len(df_product_tags['tag_produtos_id'].unique()), '/ Total de valores na tabela', df_product_tags.shape[0])

Total de valores únicos 109 / Total de valores na tabela 109


#### AJUSTANDO - DF PRODUCTS

In [88]:
df_products.isna().sum()

id_produto                     0
id_fornecedor                  0
id_disconto                 2888
id_tipo                        1
nome                           0
ativo                          7
aprovado                       0
arquivado                      1
entregavel                     0
recuperavel                    0
tempo_preparacao_minutos       6
avaliacao_media               61
dtype: int64

In [89]:
print('Total de valores únicos', len(df_products['id_produto'].unique()), '/ Total de valores na tabela', df_products.shape[0])

Total de valores únicos 3177 / Total de valores na tabela 3177


#### JUNTANDO - DF PRODUCTS + DF PRODUCTS TAGS

In [90]:
# Juntando a tabela DF PRODUCTS com a PRODUCTS TAG
df_product_tags = df_product_tags.rename(columns={'tag_produtos_id' : 'id_tipo'})
df_products_t = pd.merge(df_products, df_product_tags, on='id_tipo', how='left')
df_products_t = df_products_t.drop('id_tipo', axis=1)
df_products_t.isna().sum()

id_produto                     0
id_fornecedor                  0
id_disconto                 2888
nome                           0
ativo                          7
aprovado                       0
arquivado                      1
entregavel                     0
recuperavel                    0
tempo_preparacao_minutos       6
avaliacao_media               61
destacado                      1
texto_display                  1
tipo                           1
dtype: int64

In [91]:
df_products_t.shape

(3177, 14)

In [92]:
dic = {'id_produto' : 'id_produto', 
       'id_fornecedor' : 'id_fornecedor', 
       'id_disconto' : 'id_desconto', 
       'nome' : 'prod_nome', 
       'ativo' : 'prod_ativo',
       'aprovado' : 'prod_aprovado', 
       'arquivado' : 'prod_arquivado', 
       'entregavel' : 'prod_entregavel', 
       'recuperavel' : 'prod_recuperavel',
       'tempo_preparacao_minutos' : 'prod_tempo_prep_minuto', 
       'avaliacao_media' : 'prod_avaliacao_media', 
       'destacado' : 'prod_tag_destacado',
       'texto_display' : 'prod_tag_texto_display', 
       'tipo' : 'prod_tag_tipo'}
df_products_t = df_products_t.rename(columns=dic)
df_products_t.columns

Index(['id_produto', 'id_fornecedor', 'id_desconto', 'prod_nome', 'prod_ativo',
       'prod_aprovado', 'prod_arquivado', 'prod_entregavel',
       'prod_recuperavel', 'prod_tempo_prep_minuto', 'prod_avaliacao_media',
       'prod_tag_destacado', 'prod_tag_texto_display', 'prod_tag_tipo'],
      dtype='object')

In [93]:
# Juntando a tabela DF PRODUCTS T com a DF VENDORS T
df_products_t = pd.merge(df_products_t, df_vendors_t, on='id_fornecedor', how='left')
df_products_t = df_products_t.drop('id_fornecedor', axis=1)
df_products_t.isna().sum()

id_produto                                   0
id_desconto                               2888
prod_nome                                    0
prod_ativo                                   7
prod_aprovado                                0
prod_arquivado                               1
prod_entregavel                              0
prod_recuperavel                             0
prod_tempo_prep_minuto                       6
prod_avaliacao_media                        61
prod_tag_destacado                           1
prod_tag_texto_display                       1
prod_tag_tipo                                1
nome_fornecedor                              0
forc_data_registro                           0
forc_e_aprovado                              0
forc_delivery_raio_km                        0
forc_delivery_sistema                     2532
forc_hora_abertura                           0
forc_hora_fechamento                         0
forc_data_aprovacao                        153
forc_data_rep

In [94]:
df_products_t.shape

(3177, 40)

### DF USERS

In [95]:
# Sugestão: ainda não fazer merge com a tabela de fornecedores para não poluir a quantidade de colunas
id_fornecedor = df_users[~df_users['id_fornecedor'].isna()]['id_fornecedor'].tolist()
print('Usuários que são também vendedores: ', df_vendors[df_vendors['id_fornecedor'].isin(id_fornecedor)].shape[0])

Usuários que são também vendedores:  403


In [96]:
dic = {'id_usuario': 'id_usuario', 
       'id_fornecedor': 'user_id_fornecedor', 
       'nome_completo': 'user_nome_completo', 
       'data_ultima_compra': 'user_data_ultima_compra',
       'qtde_total_compra': 'user_qtde_total_compra', 
       'gasto_total': 'user_gasto_total',
       'ultima_qtde_compra': 'user_ultima_qtde_compra',
       'ticket_medio': 'user_ticket_medio',
       'aceita_thrid_party_mkt': 'user_aceita_thrid_party_mkt',
       'aceita_mkt': 'user_aceita_mkt'}
df_users = df_users.rename(columns=dic)
df_users.columns

Index(['id_usuario', 'user_id_fornecedor', 'user_nome_completo',
       'user_data_ultima_compra', 'user_qtde_total_compra', 'user_gasto_total',
       'user_ultima_qtde_compra', 'user_ticket_medio',
       'user_aceita_thrid_party_mkt', 'user_aceita_mkt'],
      dtype='object')

### DF CUSTOMER PAYMENTS T = DF CUSTOMER PAYMENTS + DF USERS

#### AJUSTANDO DF CUSTOMER PAYMENTS

In [97]:
df_customer_payments.isna().sum()

id_ordens                0
id_pagamentos_cliente    0
id_cliente               0
data_pagamento           0
valor                    0
status_pagamento         0
metodo_pagamento         0
dtype: int64

In [98]:
print('Total de valores únicos', len(df_customer_payments['id_pagamentos_cliente'].unique()), '/ Total de valores na tabela', df_customer_payments.shape[0])

Total de valores únicos 7277 / Total de valores na tabela 8295


In [111]:
df_customer_payments[['id_pagamentos_cliente', 'id_ordens']].groupby('id_pagamentos_cliente').count().sort_values('id_ordens', ascending=False).head(1500)

,id_ordens
id_pagamentos_cliente,
5fc156d29ce0e20021fdbbda,8
5fbefb467dcc570021f3b278,6
5fbd934f5da74a0021e6129d,6
5fc14ded9ce0e20021fd6ae7,6
605531fce5b3670021fa3451,6
60f188e28ef4ff0021c170e1,5
6041424a317f730021f5b369,5
60f188998ef4ff0021c17042,5
5fbeb8de1c97050021f9766a,5


In [99]:
print('Total de valores únicos', len(df_customer_payments['id_ordens'].unique()), '/ Total de valores na tabela', df_customer_payments.shape[0])

Total de valores únicos 5688 / Total de valores na tabela 8295


In [100]:
print('Total de valores únicos', len(df_customer_payments['id_cliente'].unique()), '/ Total de valores na tabela', df_customer_payments.shape[0])

Total de valores únicos 3076 / Total de valores na tabela 8295


In [10]:
dic = {'id_ordens' : 'id_ordem', 
       'id_pagamentos_cliente' : 'id_pagamentos_cliente', 
       'id_cliente' : 'id_usuario', 
       'data_pagamento' : 'user_pay__data_pagamento',
       'valor' : 'user_pay_valor', 
       'status_pagamento': 'user_pay_status_pagamento', 
       'metodo_pagamento': 'user_pay_metodo_pagamento'}
# Juntando DF CUSTOMER PAYMENTS com a DF USERS
df_customer_payments = df_customer_payments.rename(columns=dic)
df_customer_payments_t = pd.merge(df_customer_payments, df_users, on='id_usuario', how='left')
df_customer_payments_t = df_customer_payments_t.drop('id_usuario', axis=1)
# Não faz sentido manter se o usuário é também fornecedor para esta análise (poderá ser revisada posteriormente)
df_customer_payments_t = df_customer_payments_t.drop('user_id_fornecedor', axis=1)
df_customer_payments_t.isna().sum()

id_ordem                          0
id_pagamentos_cliente             0
user_pay__data_pagamento          0
user_pay_valor                    0
user_pay_status_pagamento         0
user_pay_metodo_pagamento         0
user_nome_completo                0
user_data_ultima_compra        1936
user_qtde_total_compra            1
user_gasto_total                  1
user_ultima_qtde_compra           1
user_ticket_medio                 1
user_aceita_thrid_party_mkt       1
user_aceita_mkt                   1
dtype: int64

### <font color='red'>DF ORDERED PRODUCTS T =  DF ORDERED PRODUCTS + DF PRODUCTS T </font>

**Tabela com os dados vendidos pela plataforma onde está vinculado:**
 - a ordem do pedido com as respectivas informações
 - o nome do produto e os respectivos dados de cadastro
 - o nome do fornecedor e os respectivos dados de cadastro
 - dados do cliente que realizou a compra

In [26]:
# JUNTANDO DF ORDERED PRODUCTS com DF PRODUCS T
dic = {'id_produto_vendido': 'id_produto_vendido', 
       'id_produto': 'id_produto', 
       'id_variacao_selecionada' : 'id_variacao_selecionada',
       'valor_medida': 'order_prod_valor_medida', 
       'tipo_medida' : 'order_prod_tipo_medida', 
       'numero_pessoas_servidas' : 'order_prod_numero_pessoas_servidas', 
       'preco' : 'order_prod_preco',
       'qtde' : 'order_prod_qtde', 
       'anotacao' : 'order_prod_anotacao', 
       'subtotal' : 'order_prod_subtotal', 
       'id_ordem' : 'id_ordem'}
df_ordered_products = df_ordered_products.rename(columns=dic)
df_ordered_products_t = pd.merge(df_ordered_products, df_products_t, on='id_produto', how='left')
df_ordered_products_t = df_ordered_products_t.drop('id_produto', axis=1)
df_ordered_products_t.isna().sum()

id_produto_vendido                               0
id_variacao_selecionada                        146
order_prod_valor_medida                        104
order_prod_tipo_medida                         104
order_prod_numero_pessoas_servidas             106
order_prod_preco                               148
order_prod_qtde                                148
order_prod_anotacao                          32317
order_prod_subtotal                            234
id_ordem                                       234
id_desconto                                  26295
prod_nome                                      162
prod_ativo                                     197
prod_aprovado                                  162
prod_arquivado                                 162
prod_entregavel                                162
prod_recuperavel                               162
prod_tempo_prep_minuto                         197
prod_avaliacao_media                           201
prod_tag_destacado             

In [27]:
df_ordered_products_t = pd.merge(df_ordered_products_t, df_customer_payments_t, on='id_ordem', how='left')
df_ordered_products_t = df_ordered_products_t.drop('id_ordem', axis=1)
df_ordered_products_t.isna().sum()

id_produto_vendido                               0
id_variacao_selecionada                        146
order_prod_valor_medida                        104
order_prod_tipo_medida                         104
order_prod_numero_pessoas_servidas             106
order_prod_preco                               148
order_prod_qtde                                148
order_prod_anotacao                          37229
order_prod_subtotal                            234
id_desconto                                  30307
prod_nome                                      162
prod_ativo                                     203
prod_aprovado                                  162
prod_arquivado                                 162
prod_entregavel                                162
prod_recuperavel                               162
prod_tempo_prep_minuto                         203
prod_avaliacao_media                           207
prod_tag_destacado                             168
prod_tag_texto_display         

In [28]:
df_ordered_products_t.shape

(39189, 61)

# <font color='red'> 1.0. Descrição dos dados </font>

In [4]:
df_solds_cognatis.head()

,SETOR_ATIV,UF,MUNICIPIO,CEP,TRABALHADORES,S_RENDA_MEDIA,S_FLUXO_PEDESTRE,S_FLUXO_VEICULOS,S_POP_TOTAL,S_DOM_PART_PERM,S_PCT_CRIANCAS,S_PCT_ADOLESCENTE,S_PCT_JOVENS,S_PCT_ADULTOS,S_PCT_SENIOR,S_PCT_IDOSOS,S_RENDA_MEDIA_FAMILIAR,S_DENSIDADE_POPULACAO_KM2,S_PCT_CLASSE_A,S_PCT_CLASSE_B,S_PCT_CLASSE_C,S_PCT_CLASSE_D,S_PCT_CLASSE_E,S_TX_CRESC_POP,S_TX_CRESC_DOM,S_TX_CRESC_RENDA_MEDIA,S_RENDA_PER_CAPITA,S_COMERCIALIZACAO,S_QTD_EMPRESAS,S_PCT_CNAE_AGRICOLA,S_PCT_CNAE_COMERCIO,S_PCT_CNAE_CONSTR_CIVIL,S_PCT_CNAE_INDUSTRIA,S_PCT_CNAE_SERVICOS,S_PCT_CNAE_OUTROS,S_SEM_FUNC,S_DE_1_4_FUNC,S_DE_5_9_FUNC,S_DE_10_19_FUNC,S_DE_20_49_FUNC,S_DE_50_99_FUNC,S_DE_100_249_FUNC,S_DE_250_499_FUNC,S_DE_500_999_FUNC,S_MAIS1000_FUNC,S_SUPERMERCADO,S_TT_PADARIA,S_ATACADO,S_HIPERMERCADO,S_MINIMERCADO
0,NaN,NaN,NaN,NaN,18776.596364,1852.325903,3.158788,4.933333,61660.275152,22297.529697,12.024473,13.205539,15.781139,31.952715,17.218048,9.818206,5372.332256,5057.621174,5.607006,31.245418,40.717103,19.330109,3.100036,1.702618,3.412485,1.346521,2028.925312,14.765867,7202.386667,0.814317,32.400931,6.790309,8.031730,51.080748,0.500792,4817.477576,708.803636,244.792727,135.844848,79.586667,22.883636,12.442424,4.026667,2.001212,1.670303,8.768485,57.606061,23.126061,2.865455,85.970909
1,NaN,AC,Rio Branco,69900001.0,13125.000000,3502.669922,4.000000,6.000000,53376.000000,17146.000000,15.950000,17.559999,18.250000,30.020000,12.170000,6.050000,2121.070068,4036.810059,0.610000,10.160000,39.750000,40.520000,8.960000,1.440000,3.030000,0.730000,684.109985,9.480000,3624.000000,0.607064,45.226269,4.746137,7.008830,40.590508,1.820000,2757.000000,324.000000,101.000000,71.000000,42.000000,15.000000,15.000000,3.000000,4.000000,2.000000,3.000000,22.000000,3.000000,9.000000,184.000000
2,NaN,AC,Rio Branco,69900001.0,18364.000000,2477.159912,4.000000,6.000000,73460.000000,24698.000000,12.570000,15.060000,17.820000,32.400002,15.480000,6.680000,4266.919922,6317.299805,4.440000,27.270000,40.430000,24.360001,3.490000,1.340000,3.100000,0.780000,1438.170044,12.000000,4795.000000,0.312826,42.168926,6.068822,5.505735,44.671533,1.270000,3295.000000,540.000000,190.000000,131.000000,65.000000,19.000000,19.000000,6.000000,2.000000,2.000000,14.000000,29.000000,27.000000,9.000000,111.000000
3,NaN,AC,Sena Madureira,NaN,960.000000,1225.869995,0.000000,0.000000,26675.500000,7823.500000,14.165000,20.740000,21.139999,29.105000,9.555000,5.300000,1917.375000,46.920000,0.185000,7.225000,26.474999,37.225000,28.889999,3.515000,5.265000,1.840000,564.924988,7.145000,951.500000,0.708593,55.977584,5.074720,6.833748,30.009963,1.400000,708.000000,100.500000,27.000000,12.000000,3.000000,1.000000,0.000000,0.000000,0.500000,0.000000,3.000000,7.500000,5.000000,1.000000,84.000000
4,NaN,AC,Senador Guiomard,NaN,1102.000000,1272.079956,0.000000,1.000000,11183.000000,3732.000000,17.360001,18.510000,19.440001,27.600000,11.130000,5.960000,2074.469971,119.680000,0.270000,10.240000,31.000000,44.160000,14.330000,2.410000,4.130000,1.100000,693.349976,6.180000,559.000000,1.073345,52.415027,5.545617,7.513417,32.558140,0.890000,423.000000,49.000000,14.000000,5.000000,6.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,10.000000,0.000000,0.000000,42.000000
